In [1]:
import os
import bs4
import json
import time
import lxml
import requests
import cloudscraper
from pathlib import Path
from random import randint
from fake_useragent import UserAgent
from IPython.display import clear_output as clear

In [2]:
ua = UserAgent()

In [3]:
top_dir = os.path.abspath('').replace('.corrections', '')
conjugations_data_dir = os.path.join(top_dir, 'conjugations', 'data')
missing_dir = os.path.join(top_dir, '.corrections', 'out')
out_dir = os.path.join(top_dir, '.correcitions', 'out')

In [4]:
headers = {"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
           "accept-encoding": "gzip, deflate, br",
           "accept-language": "en-GB,en;q=0.9,es-ES;q=0.8,es;q=0.7,en-US;q=0.6,eu;q=0.5",
           "cache-control": "max-age=0",
           "cookie": "t=238707487; _ga=GA1.2.1376835774.1641262578; _gid=GA1.2.1482423077.1641262578; _fbp=fb.1.1641262579526.851471446",
           "referer": "https://hidemy.name/en/proxy-list/?start=64",
           "sec-ch-ua-mobile": "?0",
           "sec-ch-ua-platform": "macOS",
           "sec-fetch-dest": "document",
           "sec-fetch-mode": "navigate",
           "sec-fetch-site": "same-origin",
           "sec-fetch-user": "?1",
           "upgrade-insecure-requests": "1",
           "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"}

In [ ]:
def extractText(string):
    skip = 0
    output = ''
    
    for char in string:
        if char != '(' and char != ')':
            if skip == 0:
                output += char
        else:
            if char == '(':
                skip += 1
            if char == ')':
                skip -= 1
    
    return output.strip().replace('  ', ' ')

In [ ]:
def getCollinsTranslations(verb):
    
    retries = 1
    equivalent = ''

    while True:
        try:
            url = f'https://www.collinsdictionary.com/dictionary/{language}-english/{verb}'
            scraper = cloudscraper.create_scraper(
                browser={
                    'browser': 'firefox',
                    'platform': 'windows',
                    'mobile': False
                })
            soup = bs4.BeautifulSoup(scraper.get(url, headers = {"useragent": f"{ua.random}"}).text, 'lxml')

            if 'Cloudflare' not in soup.select('title')[0].getText():
                translations = []

                for section in soup.select('.page .dictionary .hom'):
                    if section.select('.hi.rend-sc .pos'):
                        section_type = section.select('.hi.rend-sc .pos')[0].getText().lower()
                    elif section.select('.gramGrp .pos'):
                        section_type = section.select('.gramGrp .pos')[0].getText().lower()
                    else:
                        section_type = False
                    
                        if section.select('.sense .xr a.ref'):
                            if 'Translation of' in str(section.select('.sense .xr a.ref')[0]):
                                equivalent = section.select('.sense .xr a.ref')[0].getText()
                    
                    if section_type:
                        if 'verb' in section_type and 'adverb' not in section_type and 'reflexive' not in section_type:
                            for entry in section.select('.sense:not(.type-example)>.cit.type-translation .quote'):
                                
                                translation = entry.getText().strip()

                                if entry.select('.or.i'):
                                    translation = translation.split(' or ')[0]  

                                if translation[:3] == 'to ':
                                    translation = translation[3:]
                                
                                    if translation.replace(' ','').isalpha() and translation not in translations:
                                        translations.append(translation)

                if translations:
                    return translations
                else:
                    return equivalent
            
            else:
                retries += 1
        
        except:
            retries += 1

        if retries > 5:
            return []     

In [ ]:
def getCambridgeTranslations(verb):
    url = f'https://dictionary.cambridge.org/dictionary/{language}-english/{verb}'
    page = requests.get(url, headers = headers)
    soup = bs4.BeautifulSoup(page.text, 'lxml')

    translations = []

    for main in soup.select('.pr.dictionary'):
        try: 
            if main.select('.pos.dpos')[0].getText() == 'verb':
                    for item in main.select('.di-body.normal-entry-body .pr:not(.phrase-block)')[0].select('.trans.dtrans:not(.hdb)'):
                        translation = item.getText()
                        
                        if translation[-1] == ')' and translation.count('(') == 1:
                            translation = translation.split('(')[0].strip()
                
                        if translation[:3] == 'to ' and translation[3:].replace(' ','').isalpha() and translation[3:] not in translations:
                            translations.append(translation[3:].replace(' sth', ' something').replace(' sb', ' somebody'))
        except:
            pass

        try:
            for main in soup.select('.pr.dictionary'):
                if main.select('h2.c_hh'):
                    for block in main.select('.di-body.normal-entry-body .pr:not(.phrase-block)'):
                        
                        item = block.select('.def-body.ddef_b.ddef_b-t')[0]
                        word_class = item.getText().split('[')[1].split(']')[0].strip()

                        if word_class == 'verb':
                            
                            generic = item.getText().split('[')[0].strip()

                            if generic.replace(' ','').isalpha() and generic not in translations:
                                translations.append(generic.replace(' sth', ' something').replace(' sb', ' somebody'))

                            detail = item.getText().split(']')[1].strip()

                            for translation in detail.split(';'):
                                formatted = extractText(translation.strip())
                                
                                if formatted[:3] == 'to ':
                                    candidate = formatted[3:]

                                    if len(candidate.split(' ')) == 1:
                                        translations.append(candidate.replace(' sth', ' something').replace(' sb', ' somebody'))
                                        break

                                    if ' or ' in candidate and len(candidate.split(' ')) == 3:
                                        for c in candidate.split(' or '):
                                            if c not in translations:
                                                translations.append(c.replace(' sth', ' something').replace(' sb', ' somebody'))
        except:
            pass

    return translations[:10]

In [ ]:
def getSpanishDictTranslations(verb):
    output = []

    request = requests.get(f"https://www.spanishdict.com/translate/{verb}")
    soup = bs4.BeautifulSoup(request.text, "lxml")

    if not soup.select('._25QSB23Y'):

        for i in range(0, len(soup.select("._2vd6M2gR"))):
            entry = soup.select("._2vd6M2gR")[i].getText()

            if entry[0:3] == "to " and entry[3:] not in output:
                if entry[3:].replace(' ','').isalpha():
                    output.append(entry[3:])

        for i in range(0, len(soup.select(".gram_cat"))):
            block = soup.select(".gram_cat")[i]

            if verb + 'se' not in block.getText():

                for i in range(0, len(block.select(".tran_main"))):
                    entry = block.select(".tran_main")[i].getText()

                    if entry[0:3] == "to " and entry[3:] not in output:
                        if entry[3:].replace(' ','').isalpha():
                            output.append(entry[3:])
        
    return output

In [5]:
language = 'spanish'

In [6]:
with open(os.path.join(conjugations_data_dir, 'intermediates', f'infinitives_{language}_intermediate.json'), 'r', encoding = 'utf8') as file:
    infinitives = json.loads(file.read())

with open(os.path.join(conjugations_data_dir, 'language-specific', f'conjugations_{language}.json'), 'r', encoding = 'utf8') as file:
    conjugations = json.loads(file.read())

In [7]:
missing = []

for infinitive in infinitives:
    if infinitive[0] not in conjugations and infinitive[1] <= 2000:
        missing.append(infinitive)

In [ ]:
data = {}

for verb in missing:

    infinitive = verb[0]

    try:
        collins = getCollinsTranslations(infinitive)

        if isinstance(collins, str):
            collins = getCollinsTranslations(collins)
            
        cambridge = getCambridgeTranslations(infinitive)

        if collins:
            output = collins

            for translation in cambridge:
                if translation not in output:
                    output.append(translation)

        else:
            output = cambridge

        data[infinitive] = output[:10]

        clear(); print(f'{verb} - complete')

    except Exception as ex:
        data[infinitive] = []
        clear(); print(f'{verb} - error ({ex})')
        
    time.sleep(randint(1000, 2000) / 10000)

if language == 'spanish':

    combined = {}

    for verb in data:
        spanishdict = getSpanishDictTranslations(verb)
        translations = data[verb][::-1]

        output = spanishdict

        for n in range(0, len(translations)):
            
            if translations[n] not in output:
                iterate = True
                x = n

                while iterate:
                    if translations[x] in output:
                        for i in range(0, len(output)):
                            if output[i] == translations[x]:
                                output.insert(i + 1, translations[n])
                                iterate = False
                                break
                    else:
                        if x == 0:
                            iterate = False
                        else:
                            x -= 1
            
            if translations[n] not in output:
                output.append(translations[n])

        combined[verb] = output[:10]
        
        clear(); print(f'{verb} - complete')

In [ ]:
for entry,verb in zip(missing, data):
    if data[verb]:
        print(entry, data[verb])

In [8]:
retain = {
    'spanish':  ['sustituir', 'aportar'],
    
    'french':   [],
    
    'german':   ['geschehen', 'helfen', 'berufen', 'verhalten', 'vorbehalten', 'beruhen', 'überlegen', 'bauen', 
                'erwerben', 'sterben', 'werfen', 'verfahren', 'verborgen', 'rücken', 'unterwerfen', 'entwerfen', 
                'ausreichen', 'bescheiden', 'verlegen', 'blicken', 'werben', 'unterlegen', 'frischen', 'beten', 
                'verwerfen', 'eindringen', 'nahelegen', 'gedeihen', 'bewerben', 'aufwerfen', 'verderben', 'arrangieren', 
                'bequemen', 'entgegenkommen', 'stehlen', 'beharren', 'zusammenwirken', 'blühen', 'benehmen', 'begaben', 
                'mithelfen', 'verwehren', 'anklingen', 'weiterhelfen', 'hinzutreten', 'gönnen', 'unterblieben', 'hinwirken', 
                'verschwimmen', 'verschlagen', 'zutrauen', 'einfordern', 'ermächtigen', 'entgegenbringen', 'durchringen', 
                'betrauen', 'erschöpften', 'ausersehen', 'rezipieren', 'ausklammern'],

    'italian':  ['usare', 'avvenire', 'risultare', 'sembrare', 'accadere', 'succedere', 'convenire', 'calcare', 'rischiare',
                'dispiacere', 'bastare', 'capitare', 'discernere', 'importare', 'scolare', 'propendere', 'decadere', 'balenare'],
                
    'portuguese': []
}

In [9]:
output = [verb for verb in missing if verb[0] in retain[language]]

In [10]:
len(output), len(retain[language])

(2, 2)

In [11]:
f = os.path.join(missing_dir, f'missing_selected_{language}.json')

with open(f, "w", encoding = 'utf8') as file:
    json.dump(output, file, indent = 4, ensure_ascii = False)